In [17]:
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, SpatialDropout1D,GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
import matplotlib as plt

# NLTK
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

# Other
import re
import string
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE

In [18]:
import numpy as np
from numpy import asarray
import pandas as pd

from bs4 import BeautifulSoup 
import re 
from nltk.corpus import stopwords
import nltk
from sklearn.preprocessing import OneHotEncoder
import keras 
from keras.models import Sequential, Model 
from keras import layers
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, Input, Embedding
from keras.layers.merge import Concatenate
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [19]:
def load_lines(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.readlines()
	# close the file
	file.close()
	text = [x.strip() for x in text] 
	while '' in text:
	  text.remove('')
	return text

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

In [20]:
x_train= load_lines("/home2/satyamishra/DLNLP/convlstm/OneDrive_3_04-11-2020/SST-2.x.train.txt")
y_train = load_lines("/home2/satyamishra/DLNLP/convlstm/OneDrive_3_04-11-2020/SST-2.y.train.txt")
x_dev= load_lines("/home2/satyamishra/DLNLP/convlstm/OneDrive_3_04-11-2020/SST-2.x.dev.txt")
y_dev = load_lines("/home2/satyamishra/DLNLP/convlstm/OneDrive_3_04-11-2020/SST-2.y.dev.txt")


In [21]:
def load_embedding(filename):
	# load embedding into memory, skip first line
	file = open(filename,'r')
	lines = file.readlines()[1:]
	file.close()
	# create a map of words to vectors
	embedding = dict()
	for line in lines:
		parts = line.split()
		# key is string word, value is numpy array for vector
		embedding[parts[0]] = asarray(parts[1:], dtype='float32')
	return embedding
#raw_embedding is a dictionary
embeddings_index = load_embedding('/home2/satyamishra/DLNLP/glove.6B/glove.6B.200d.txt')

In [22]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [23]:
tokenizer = Tokenizer(num_words=100000)
tokenizer.fit_on_texts(x_train)
sequences = tokenizer.texts_to_sequences(x_train)

In [24]:
length = []
for x in x_train:
    length.append(len(x.split()))
max(length)

52

In [ ]:
x_train_seq = pad_sequences(sequences, maxlen=60)


In [ ]:
sequences_val = tokenizer.texts_to_sequences(x_dev)
x_val_seq = pad_sequences(sequences_val, maxlen=60)

In [ ]:
num_words = 100000
embedding_matrix = np.zeros((num_words, 200))
for word, i in tokenizer.word_index.items():
    if i >= num_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y_train)
y_train=le.transform(y_train).ravel()
y_dev=le.transform(y_dev).ravel()


In [ ]:
from keras.layers import Input, Dense, concatenate, Activation
from keras.models import Model
tweet_input = Input(shape=(60,), dtype='int32')

tweet_encoder = Embedding(100000, 200, weights=[embedding_matrix], input_length=60, trainable=True)(tweet_input)
bigram_branch = Conv1D(filters=100, kernel_size=2, padding='valid', activation='relu', strides=1)(tweet_encoder)
bigram_branch = GlobalMaxPooling1D()(bigram_branch)
trigram_branch = Conv1D(filters=100, kernel_size=3, padding='valid', activation='relu', strides=1)(tweet_encoder)
trigram_branch = GlobalMaxPooling1D()(trigram_branch)
fourgram_branch = Conv1D(filters=100, kernel_size=4, padding='valid', activation='relu', strides=1)(tweet_encoder)
fourgram_branch = GlobalMaxPooling1D()(fourgram_branch)
merged = concatenate([bigram_branch, trigram_branch, fourgram_branch], axis=1)

merged = Dense(256, activation='relu')(merged)
merged = Dropout(0.5)(merged)
merged = Dense(1)(merged)
output = Activation('sigmoid')(merged)
model = Model(inputs=[tweet_input], outputs=[output])
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 60)           0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 60, 200)      20000000    input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 59, 100)      40100       embedding_2[0][0]                
__________________________________________________________________________________________________
conv1d_5 (Conv1D)               (None, 58, 100)      60100       embedding_2[0][0]                
____________________________________________________________________________________________

In [ ]:


model.fit(x_train_seq, y_train, batch_size=32, epochs=10,
                     validation_data=(x_val_seq, y_dev))

Train on 6911 samples, validate on 872 samples
Epoch 1/10
6911/6911 [==============================] - 116s 17ms/step - loss: 0.5578 - accuracy: 0.7064 - val_loss: 0.4438 - val_accuracy: 0.7947
Epoch 2/10
6911/6911 [==============================] - 112s 16ms/step - loss: 0.2891 - accuracy: 0.8779 - val_loss: 0.4433 - val_accuracy: 0.8028
Epoch 3/10
6911/6911 [==============================] - 129s 19ms/step - loss: 0.1064 - accuracy: 0.9645 - val_loss: 0.5631 - val_accuracy: 0.8005
Epoch 4/10
6911/6911 [==============================] - 114s 17ms/step - loss: 0.0303 - accuracy: 0.9920 - val_loss: 0.7066 - val_accuracy: 0.8073
Epoch 5/10
6911/6911 [==============================] - 119s 17ms/step - loss: 0.0103 - accuracy: 0.9973 - val_loss: 0.8040 - val_accuracy: 0.8096
Epoch 6/10
6911/6911 [==============================] - 116s 17ms/step - loss: 0.0029 - accuracy: 0.9999 - val_loss: 0.8778 - val_accuracy: 0.8200
Epoch 7/10
6911/6911 [==============================] - 117s 17ms/step 